In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools
import os
import shutil

import adanet
import tensorflow as tf

# The random seed to use.
RANDOM_SEED = 42

LOG_DIR = '/tmp/overfitmodels'

W1229 00:51:25.710059 139856257656576 module_wrapper.py:139] From /root/anaconda3/lib/python3.6/site-packages/adanet/tf_compat/__init__.py:87: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

W1229 00:51:25.733109 139856257656576 module_wrapper.py:139] From /root/anaconda3/lib/python3.6/site-packages/adanet/core/tpu_estimator.py:39: The name tf.estimator.tpu.TPUEstimator is deprecated. Please use tf.compat.v1.estimator.tpu.TPUEstimator instead.



## [Medium 글](https://towardsdatascience.com/googles-adanet-uses-neural-networks-to-build-better-neural-networks-36c07eefd4d3)

## [Github TUTORIAL](https://github.com/tensorflow/adanet/blob/master/adanet/examples/tutorials/adanet_objective.ipynb)

In [2]:
tf.__version__

'1.15.0'

In [6]:
#@test {"skip": true}

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# Install ngrok binary.
# ! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# ! unzip ngrok-stable-linux-amd64.zip

# Delete old logs dir.
shutil.rmtree(LOG_DIR, ignore_errors=True)

# print("Follow this link to open TensorBoard in a new tab.")
# get_ipython().system_raw('./ngrok http 6006 &')
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [7]:
from sklearn.model_selection import train_test_split 
import pandas as pd

In [8]:
data = pd.read_csv("./Data/DontOverfit/train.csv")
test_data = pd.read_csv("./Data/DontOverfit/test.csv")
__id__ = data.pop("id")
target = data.pop("target")
x_train , x_test , y_train  , y_test = train_test_split(data.values , target.values)

# Preview the first example from the training data
print('Model inputs: %s \n' % x_train[0])
print('Model output: %s ' % (y_train[0]))

Model inputs: [-0.838  0.28   1.171  1.627  1.826  2.1    0.14  -1.102  0.441  0.435
 -0.485 -1.899 -0.989  1.037  0.296  0.317 -1.004 -1.096  0.094  1.307
 -0.783  0.434 -0.173 -0.606  0.275 -0.221 -1.709 -0.157  0.598 -1.561
 -0.308  0.302  0.267  0.501  0.297  0.312  1.433 -1.551  0.708 -0.941
  0.459 -0.202  0.832  0.757  1.271  1.009  0.555 -0.348  0.087 -0.704
 -1.842 -1.005  0.987 -0.207 -2.345  1.338  1.137  1.568 -0.924 -0.014
  0.033 -0.928  0.453  0.89   0.426  1.87  -0.464  0.058 -0.134 -0.824
  0.302 -1.958  0.917 -1.827 -1.216 -0.619 -0.538 -2.183  0.259 -1.029
  0.217 -0.152  0.616 -2.246  1.278 -0.982 -1.75  -1.312 -0.77   1.702
 -0.214 -1.269  0.855  0.47  -0.004 -0.678  2.551 -1.582  1.085 -0.624
 -0.018  0.183  0.305  0.854 -0.361  1.16  -0.299 -1.398 -0.599 -0.481
 -1.06  -0.268 -0.112  0.637  0.955  2.104  0.451 -0.263  0.851 -0.085
  0.406  0.936 -0.992 -0.228  1.752  1.277  0.358  0.934  1.33   1.086
 -0.118 -1.754  1.653  0.419  1.16  -0.692  0.577  1.307  0.408

In [9]:
x_train.shape , x_test.shape

((187, 300), (63, 300))

In [18]:
INPUT_DIM = x_train.shape[1]
print(INPUT_DIM)

300


In [11]:
FEATURES_KEY = "x"

def input_fn(partition, training, batch_size):
    """Generate an input function for the Estimator."""
    def _input_fn():
        if partition == "train":
            dataset = tf.data.Dataset.from_tensor_slices(({
                FEATURES_KEY: x_train}, y_train))
        else:
            dataset = tf.data.Dataset.from_tensor_slices(({
                FEATURES_KEY:x_test }, y_test ))
    # We call repeat after shuffling, rather than before, to prevent separate
    # epochs from blending together.
        if training:
            dataset = dataset.shuffle(10 * batch_size, seed=RANDOM_SEED).repeat()

        dataset = dataset.batch(batch_size)
        iterator = dataset.make_one_shot_iterator()
        features, labels = iterator.get_next()
        return features, labels

    return _input_fn

In [12]:
_NUM_LAYERS_KEY = "num_layers"


class _SimpleDNNBuilder(adanet.subnetwork.Builder):
  """Builds a DNN subnetwork for AdaNet."""

  def __init__(self, optimizer, layer_size, num_layers, learn_mixture_weights,
               seed):
    """Initializes a `_DNNBuilder`.

    Args:
      optimizer: An `Optimizer` instance for training both the subnetwork and
        the mixture weights.
      layer_size: The number of nodes to output at each hidden layer.
      num_layers: The number of hidden layers.
      learn_mixture_weights: Whether to solve a learning problem to find the
        best mixture weights, or use their default value according to the
        mixture weight type. When `False`, the subnetworks will return a no_op
        for the mixture weight train op.
      seed: A random seed.

    Returns:
      An instance of `_SimpleDNNBuilder`.
    """

    self._optimizer = optimizer
    self._layer_size = layer_size
    self._num_layers = num_layers
    self._learn_mixture_weights = learn_mixture_weights
    self._seed = seed

  def build_subnetwork(self,
                       features,
                       logits_dimension,
                       training,
                       iteration_step,
                       summary,
                       previous_ensemble=None):
    """See `adanet.subnetwork.Builder`."""

    input_layer = tf.to_float(features[FEATURES_KEY])
    kernel_initializer = tf.glorot_uniform_initializer(seed=self._seed)
    last_layer = input_layer
    for _ in range(self._num_layers):
      last_layer = tf.layers.dense(
          last_layer,
          units=self._layer_size,
          activation=tf.nn.relu,
          kernel_initializer=kernel_initializer)
    logits = tf.layers.dense(
        last_layer,
        units=logits_dimension,
        kernel_initializer=kernel_initializer)

    persisted_tensors = {_NUM_LAYERS_KEY: tf.constant(self._num_layers)}
    return adanet.Subnetwork(
        last_layer=last_layer,
        logits=logits,
        complexity=self._measure_complexity(),
        persisted_tensors=persisted_tensors)

  def _measure_complexity(self):
    """Approximates Rademacher complexity as the square-root of the depth."""
    return tf.sqrt(tf.to_float(self._num_layers))

  def build_subnetwork_train_op(self, subnetwork, loss, var_list, labels,
                                iteration_step, summary, previous_ensemble):
    """See `adanet.subnetwork.Builder`."""
    return self._optimizer.minimize(loss=loss, var_list=var_list)

  def build_mixture_weights_train_op(self, loss, var_list, logits, labels,
                                     iteration_step, summary):
    """See `adanet.subnetwork.Builder`."""

    if not self._learn_mixture_weights:
      return tf.no_op()
    return self._optimizer.minimize(loss=loss, var_list=var_list)

  @property
  def name(self):
    """See `adanet.subnetwork.Builder`."""

    if self._num_layers == 0:
      # A DNN with no hidden layers is a linear model.
      return "linear"
    return "{}_layer_dnn".format(self._num_layers)


class SimpleDNNGenerator(adanet.subnetwork.Generator):
  """Generates a two DNN subnetworks at each iteration.

  The first DNN has an identical shape to the most recently added subnetwork
  in `previous_ensemble`. The second has the same shape plus one more dense
  layer on top. This is similar to the adaptive network presented in Figure 2 of
  [Cortes et al. ICML 2017](https://arxiv.org/abs/1607.01097), without the
  connections to hidden layers of networks from previous iterations.
  """

  def __init__(self,
               optimizer,
               layer_size=64,
               learn_mixture_weights=False,
               seed=None):
    """Initializes a DNN `Generator`.

    Args:
      optimizer: An `Optimizer` instance for training both the subnetwork and
        the mixture weights.
      layer_size: Number of nodes in each hidden layer of the subnetwork
        candidates. Note that this parameter is ignored in a DNN with no hidden
        layers.
      learn_mixture_weights: Whether to solve a learning problem to find the
        best mixture weights, or use their default value according to the
        mixture weight type. When `False`, the subnetworks will return a no_op
        for the mixture weight train op.
      seed: A random seed.

    Returns:
      An instance of `Generator`.
    """

    self._seed = seed
    self._dnn_builder_fn = functools.partial(
        _SimpleDNNBuilder,
        optimizer=optimizer,
        layer_size=layer_size,
        learn_mixture_weights=learn_mixture_weights)

  def generate_candidates(self, previous_ensemble, iteration_number,
                          previous_ensemble_reports, all_reports):
    """See `adanet.subnetwork.Generator`."""

    num_layers = 0
    seed = self._seed
    if previous_ensemble:
      num_layers = tf.contrib.util.constant_value(
          previous_ensemble.weighted_subnetworks[
              -1].subnetwork.persisted_tensors[_NUM_LAYERS_KEY])
    if seed is not None:
      seed += iteration_number
    return [
        self._dnn_builder_fn(num_layers=num_layers, seed=seed),
        self._dnn_builder_fn(num_layers=num_layers + 1, seed=seed),
    ]

In [13]:
LEARNING_RATE = 0.001  #@param {type:"number"}
TRAIN_STEPS = 60000  #@param {type:"integer"}
BATCH_SIZE = 32  #@param {type:"integer"}
LEARN_MIXTURE_WEIGHTS = False  #@param {type:"boolean"}
ADANET_LAMBDA = 0  #@param {type:"number"}
ADANET_ITERATIONS = 3  #@param {type:"integer"}

def train_and_evaluate(experiment_name, learn_mixture_weights=LEARN_MIXTURE_WEIGHTS,
                       adanet_lambda=ADANET_LAMBDA):
    """Trains an `adanet.Estimator` to predict housing prices."""

    model_dir = os.path.join(LOG_DIR, experiment_name)
    print(model_dir)
    estimator = adanet.Estimator(
      # Since we are predicting housing prices, we'll use a regression
      # head that optimizes for MSE.
      ### change 
#      head = tf.contrib.estimator.multi_class_head(
#         NUM_CLASSES, loss_reduction=loss_reduction)
      head=tf.contrib.estimator.binary_classification_head(
          loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE),
        model_dir  = model_dir , 
      # Define the generator, which defines our search space of subnetworks
      # to train as candidates to add to the final AdaNet model.
      subnetwork_generator=SimpleDNNGenerator(
          optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE),
          learn_mixture_weights=learn_mixture_weights,
          seed=RANDOM_SEED),

      # Lambda is a the strength of complexity regularization. A larger
      # value will penalize more complex subnetworks.
      adanet_lambda=adanet_lambda,

      # The number of train steps per iteration.
      max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS,

      # The evaluator will evaluate the model on the full training set to
      # compute the overall AdaNet loss (train loss + complexity
      # regularization) to select the best candidate to include in the
      # final AdaNet model.
      evaluator=adanet.Evaluator(
          input_fn=input_fn("train", training=False, batch_size=BATCH_SIZE)),

      # Configuration for Estimators.
      config=tf.estimator.RunConfig(
          save_summary_steps=5000,
          save_checkpoints_steps=5000,
          tf_random_seed=RANDOM_SEED,
          model_dir=model_dir) 
    )

  # Train and evaluate using using the tf.estimator tooling.
    train_spec = tf.estimator.TrainSpec(
        input_fn=input_fn("train", training=True, batch_size=BATCH_SIZE),
        max_steps=TRAIN_STEPS)
    eval_spec = tf.estimator.EvalSpec(
        input_fn=input_fn("test", training=False, batch_size=BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=30,
  )
    return [tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec) , estimator]


def ensemble_architecture(result):
    """Extracts the ensemble architecture from evaluation results."""

    architecture = result["architecture/adanet/ensembles"]
    # The architecture is a serialized Summary proto for TensorBoard.
    summary_proto = tf.summary.Summary.FromString(architecture)
    return summary_proto.value[0].tensor.string_val[0]

In [14]:
# results, _ , estimator
results = train_and_evaluate("uniform_average_classifier_ensemble_baseline")
print("Loss:", results[0]["average_loss"])
print("Architecture:", ensemble_architecture(results[0]))

W1229 00:51:58.316773 139856257656576 estimator.py:653] The following arguments have been moved to `adanet.ensemble.ComplexityRegularizedEnsembler` which can be specified in the `ensemblers` argument: ['adanet_lambda']
W1229 00:51:58.340872 139856257656576 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W1229 00:51:58.384669 139856257656576 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/data/util/random_seed.py:58: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.w

/tmp/overfitmodels/uniform_average_classifier_ensemble_baseline


W1229 00:51:58.685574 139856257656576 deprecation.py:506] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1229 00:51:59.161539 139856257656576 ensemble_builder.py:490] The `build_mixture_weights_train_op` method is deprecated. Please use the `Ensembler#build_train_op` instead.
W1229 00:51:59.361459 139856257656576 ensemble_builder.py:490] The `build_mixture_weights_train_op` method is deprecated. Please use the `Ensembler#build_train_op` instead.
W1229 00:52:41.572063 139856257656576 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/metrics_impl.py:2026: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
In

W1229 00:54:16.357116 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 00:54:17.355743 139856257656576 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.
W1229 00:54:18.822785 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 00:54:18.848654 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 00:54:19.529990 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 00:54:19.558722 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "car

W1229 00:55:20.497251 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 00:55:21.113708 139856257656576 ensemble_builder.py:490] The `build_mixture_weights_train_op` method is deprecated. Please use the `Ensembler#build_train_op` instead.
W1229 00:55:21.254051 139856257656576 ensemble_builder.py:490] The `build_mixture_weights_train_op` method is deprecated. Please use the `Ensembler#build_train_op` instead.
W1229 00:57:58.720275 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 00:57:58.744209 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 00:57:59.297030 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" ins

TypeError: tuple indices must be integers or slices, not str

In [15]:
output = results[1].predict(input_fn= input_fn("train", training=False, batch_size=1))
output = list(output)

W1229 01:02:51.530873 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:02:51.554314 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:02:52.123786 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:02:52.145721 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:02:52.871088 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:02:52.890995 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:0

In [16]:
output

[{'all_class_ids': array([0, 1], dtype=int32),
  'all_classes': array([b'0', b'1'], dtype=object),
  'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logistic': array([1.], dtype=float32),
  'logits': array([17.378635], dtype=float32),
  'probabilities': array([2.8350096e-08, 1.0000000e+00], dtype=float32)},
 {'all_class_ids': array([0, 1], dtype=int32),
  'all_classes': array([b'0', b'1'], dtype=object),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([0.], dtype=float32),
  'logits': array([-21.0938], dtype=float32),
  'probabilities': array([1.000000e+00, 6.903657e-10], dtype=float32)},
 {'all_class_ids': array([0, 1], dtype=int32),
  'all_classes': array([b'0', b'1'], dtype=object),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([0.], dtype=float32),
  'logits': array([-23.559603], dtype=float32),
  'probabilities': array([1.000000e+00, 5.864008e-11], dtype=float32)},
 {'all_clas

In [19]:
input_column = tf.feature_column.numeric_column("x", shape=(INPUT_DIM,))
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec([input_column]))

W1229 01:04:53.838853 139856257656576 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/feature_column/feature_column.py:807: NumericColumn._parse_example_spec (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


In [20]:
model_dir = '/tmp/models/uniform_average_classifier_ensemble_baseline/'
results[1].export_saved_model(export_dir_base = model_dir , 
                              serving_input_receiver_fn = serving_input_fn ,
                              checkpoint_path = None , 
                              experimental_mode='infer'
                             )

W1229 01:04:56.558268 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:04:56.576992 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:04:57.121689 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:04:57.140874 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:04:57.833308 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:04:57.853060 139856257656576 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 01:0

b'/tmp/models/uniform_average_classifier_ensemble_baseline/1577549104'

In [21]:
from tensorflow.contrib import predictor
path ="/tmp/models/uniform_average_classifier_ensemble_baseline/1577549104/"
predict_fn = predictor.from_saved_model(path)

W1229 01:05:15.166746 139856257656576 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/contrib/predictor/saved_model_predictor.py:153: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


In [22]:
predict_fn

SavedModelPredictor with feed tensors {'inputs': <tf.Tensor 'input_example_tensor:0' shape=(?,) dtype=string>} and fetch_tensors {'scores': <tf.Tensor 'adanet/iteration_2/best_export_outputs/strided_slice_3:0' shape=(?, 2) dtype=float32>, 'classes': <tf.Tensor 'adanet/iteration_2/best_export_outputs/strided_slice_4:0' shape=(?, 2) dtype=string>}

In [23]:
from tensorflow.contrib import slim
path ="/tmp/models/uniform_average_classifier_ensemble_baseline/1577549104/"
#Model = tf.saved_model.load(path, )
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.load(sess, ["serve"], export_dir= path)
    graph = tf.get_default_graph()
    print(graph.get_operations())

[<tf.Operation 'global_step/Initializer/zeros' type=Const>, <tf.Operation 'global_step' type=VariableV2>, <tf.Operation 'global_step/Assign' type=Assign>, <tf.Operation 'global_step/read' type=Identity>, <tf.Operation 'input_example_tensor' type=Placeholder>, <tf.Operation 'ParseExample/Const' type=Const>, <tf.Operation 'ParseExample/ParseExample/names' type=Const>, <tf.Operation 'ParseExample/ParseExample/dense_keys_0' type=Const>, <tf.Operation 'ParseExample/ParseExample' type=ParseExample>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/step/Initializer/zeros' type=Const>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/step' type=VariableV2>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/step/Assign' type=Assign>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/step/read' type=Identity>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/Const' type=Const>, <tf.Operation 'adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense/

In [24]:
import numpy as np

In [28]:
print(x_value)

[0.725, 1.064, 1.3330000000000002, -2.863, 0.203, 1.8980000000000001, 0.434, 1.207, -0.015, 1.459, -0.667, 0.88, -0.377, -0.914, 1.109, -1.569, -0.447, -1.159, 0.674, 0.515, 1.298, 0.797, 0.611, -0.401, 0.45, 1.295, 1.349, 0.7190000000000001, -0.8340000000000001, 0.106, -0.053, -0.03, -0.20199999999999999, 0.545, -0.914, 0.26, 0.345, 1.655, -1.875, -1.9480000000000002, 1.746, -0.14800000000000002, -0.209, 0.226, -0.05, 0.628, 1.095, 0.225, 0.191, -0.068, 1.3130000000000002, 0.755, 0.051, 1.4980000000000002, 0.23199999999999998, -1.43, -0.001, 2.588, -0.47600000000000003, 0.413, 0.9229999999999999, -1.115, 0.129, 2.095, 0.32799999999999996, 0.841, -0.816, -0.593, -0.033, -0.608, -0.358, -0.301, 1.848, 0.423, -0.726, 1.413, -0.11800000000000001, 1.556, -0.305, 1.308, 1.043, -0.147, -0.132, 0.053, -1.514, 0.741, 0.529, -0.35700000000000004, 0.038, -0.47700000000000004, 0.20800000000000002, 0.153, -0.6890000000000001, 1.669, -0.528, -0.23, -1.159, -0.9079999999999999, -0.802, 1.319, -1.396

In [29]:
exported_path = "/tmp/models/uniform_average_classifier_ensemble_baseline/1577549104/"
predictor= tf.contrib.predictor.from_saved_model(exported_path)
for x_value in x_train.tolist() : 
    model_input= tf.train.Example(features=tf.train.Features(feature={
            'x': tf.train.Feature(float_list=tf.train.FloatList(value= x_value ))        
            })) 
    model_input=model_input.SerializeToString()
    output_dict= predictor({"inputs":[model_input]})
    print(output_dict)

{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[2.8350096e-08, 1.0000000e+00]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.000000e+00, 6.903657e-10]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.000000e+00, 5.864008e-11]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 1.8054158e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 1.1771684e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 3.2251324e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 3.1612992e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 6.3925683e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 2.

{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[2.5525223e-08, 1.0000000e+00]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.000000e+00, 6.022514e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 1.5244387e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 3.2299217e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 4.8778035e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[4.4097153e-08, 1.0000000e+00]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 4.9453273e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 1.0923531e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 

In [38]:
np.argmax(output_dict["scores"][0])

array([1.0000000e+00, 6.6535455e-10], dtype=float32)

In [31]:
test_data = pd.read_csv("./Data/DontOverfit/test.csv")
test_id = test_data.pop("id")
test_id = pd.DataFrame(test_id)

In [42]:
result = []
for x_value in test_data.values.tolist() : 
    model_input= tf.train.Example(features=tf.train.Features(feature={
            'x': tf.train.Feature(float_list=tf.train.FloatList(value= x_value ))        
            })) 
    model_input=model_input.SerializeToString()
    output_dict= predictor({"inputs":[model_input]})
    result.append(output_dict["scores"][0][1])

In [43]:
test_id["target"] = result

In [48]:
test_id.to_csv("RESULT.csv", index = False)

https://stackoverflow.com/questions/52200016/tf-estimator-cant-load-pb-from-saved-model

In [79]:
with tf.Session() as sess:
    exported_path = "/tmp/models/uniform_average_classifier_ensemble_baseline/1577549104/"
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], exported_path)
    predictor= tf.contrib.predictor.from_saved_model(exported_path)
    
    for x_value in x_train.tolist() : 
        model_input= tf.train.Example(features=tf.train.Features(feature={
                'x': tf.train.Feature(float_list=tf.train.FloatList(value=x_value))        
                })) 
        model_input=model_input.SerializeToString()
        output_dict= predictor({"inputs":[model_input]})
        print(output_dict)

{'outputs': array([[149.54388]], dtype=float32)}
{'outputs': array([[144.15085]], dtype=float32)}
{'outputs': array([[246.66365]], dtype=float32)}
{'outputs': array([[127.57523]], dtype=float32)}
{'outputs': array([[239.3547]], dtype=float32)}
{'outputs': array([[141.01492]], dtype=float32)}
{'outputs': array([[241.80632]], dtype=float32)}
{'outputs': array([[164.99886]], dtype=float32)}
{'outputs': array([[166.16196]], dtype=float32)}
{'outputs': array([[169.87413]], dtype=float32)}
{'outputs': array([[244.0774]], dtype=float32)}
{'outputs': array([[203.5031]], dtype=float32)}
{'outputs': array([[151.2827]], dtype=float32)}
{'outputs': array([[243.30573]], dtype=float32)}
{'outputs': array([[127.5654]], dtype=float32)}
{'outputs': array([[198.67014]], dtype=float32)}
{'outputs': array([[245.53656]], dtype=float32)}
{'outputs': array([[140.65451]], dtype=float32)}
{'outputs': array([[149.94504]], dtype=float32)}
{'outputs': array([[153.27692]], dtype=float32)}
{'outputs': array([[190.4

{'outputs': array([[257.65842]], dtype=float32)}
{'outputs': array([[134.10489]], dtype=float32)}
{'outputs': array([[148.561]], dtype=float32)}
{'outputs': array([[228.21677]], dtype=float32)}
{'outputs': array([[138.44933]], dtype=float32)}
{'outputs': array([[132.78246]], dtype=float32)}
{'outputs': array([[167.74496]], dtype=float32)}
{'outputs': array([[196.89917]], dtype=float32)}
{'outputs': array([[117.62478]], dtype=float32)}
{'outputs': array([[164.27309]], dtype=float32)}
{'outputs': array([[144.08557]], dtype=float32)}
{'outputs': array([[242.77524]], dtype=float32)}
{'outputs': array([[136.69638]], dtype=float32)}
{'outputs': array([[139.63075]], dtype=float32)}
{'outputs': array([[151.23407]], dtype=float32)}
{'outputs': array([[144.67421]], dtype=float32)}
{'outputs': array([[242.58722]], dtype=float32)}
{'outputs': array([[163.4574]], dtype=float32)}
{'outputs': array([[149.13364]], dtype=float32)}
{'outputs': array([[140.20772]], dtype=float32)}
{'outputs': array([[147

In [23]:
path ="/tmp/models/uniform_average_ensemble_baseline/1577542872/"
#Model = tf.saved_model.load(path, )
with tf.Session(graph=tf.Graph()) as sess:
    Model = tf.saved_model.load(sess, ["serve"], export_dir= path)
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

---------
Variables: name (type shape) [size]
---------
adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense/kernel:0 (float32_ref 13x64) [832, bytes: 3328]
adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense/bias:0 (float32_ref 64) [64, bytes: 256]
adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense_1/kernel:0 (float32_ref 64x1) [64, bytes: 256]
adanet/iteration_0/subnetwork_t0_1_layer_dnn/dense_1/bias:0 (float32_ref 1) [1, bytes: 4]
adanet/iteration_0/ensemble_t0_1_layer_dnn_grow_complexity_regularized/weighted_subnetwork_0/logits/mixture_weight:0 (float32_ref ) [1, bytes: 4]
adanet/iteration_1/subnetwork_t1_2_layer_dnn/dense/kernel:0 (float32_ref 13x64) [832, bytes: 3328]
adanet/iteration_1/subnetwork_t1_2_layer_dnn/dense/bias:0 (float32_ref 64) [64, bytes: 256]
adanet/iteration_1/subnetwork_t1_2_layer_dnn/dense_1/kernel:0 (float32_ref 64x64) [4096, bytes: 16384]
adanet/iteration_1/subnetwork_t1_2_layer_dnn/dense_1/bias:0 (float32_ref 64) [64, bytes: 256]
adanet/iteration_1/subnetwork_t

In [22]:
def save_model(adanet_estimator, model_dir, input_dimension):
    def serving_input_fn():
        inputs = {
            "x": tf.placeholder(dtype=tf.float32,
                                shape=[None, input_dimension],
                                name="x")
        }
        return tf.estimator.export.ServingInputReceiver(inputs, inputs)
    return adanet_estimator.estimator.export_saved_model(model_dir, serving_input_fn)

def print_model_architechture(model_dir):
    with tf.Session(graph=tf.Graph()) as sess:
        tf.saved_model.load(sess, ["serve"], model_dir)
        model_vars = tf.trainable_variables()
        slim.model_analyzer.analyze_vars(model_vars, print_info=True)